In [1]:
import pandas as pd
import numpy as np

import os

In [8]:
# setup credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = (
    os.path.abspath("../credentials/github.txt"))

%load_ext google.cloud.bigquery

[참조 논문](https://arxiv.org/pdf/1911.00169.pdf)

![diagram](https://miro.medium.com/max/4404/1*e5NHRqew4lXNLQ7NAL7KIQ.png)

### traces

In [17]:
%%bigquery trace_df
select *
from `bigquery-public-data.crypto_ethereum.traces`
where DATE(block_timestamp) = "2021-02-16"

Downloading: 100%|██████████| 4226254/4226254 [08:20<00:00, 8446.11rows/s] 


In [18]:
trace_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4226254 entries, 0 to 4226253
Data columns (total 20 columns):
 #   Column             Dtype              
---  ------             -----              
 0   transaction_hash   object             
 1   transaction_index  float64            
 2   from_address       object             
 3   to_address         object             
 4   value              object             
 5   input              object             
 6   output             object             
 7   trace_type         object             
 8   call_type          object             
 9   reward_type        object             
 10  gas                float64            
 11  gas_used           float64            
 12  subtraces          int64              
 13  trace_address      object             
 14  error              object             
 15  status             int64              
 16  block_timestamp    datetime64[ns, UTC]
 17  block_number       int64              
 18  bl

In [6]:
trace_df.head()

,transaction_hash,transaction_index,from_address,to_address,value,input,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,block_timestamp,block_number,block_hash,trace_id
0,0x53697d964e25da59847713abec2e6b58bc86852c1368...,2,0x2f9ec37d6ccfff1cab21733bdadede11c823ccb0,0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f,0E-9,0x23b872dd000000000000000000000000000000370531...,None,call,call,None,245232.0,NaN,2,"0,1,2",Reverted,0,2021-02-16 01:46:38+00:00,11864923,0x0d0f02630d3bc2f06ed413cb51a5984a7d07d0859436...,call_0x53697d964e25da59847713abec2e6b58bc86852...
1,0x1758476271d461dfb3c2d52709f787fbd6f5648ecc55...,264,0xf97b96d0749001d65c09e21a86a2ac156be704d0,0xdac17f958d2ee523a2206206994597c13d831ec7,0E-9,0x095ea7b3000000000000000000000000f97b96d07490...,None,call,call,None,500818.0,NaN,0,"0,0,6,2",Reverted,0,2021-02-16 08:15:37+00:00,11866670,0xcad6593468eaaff533919c92ae46483eff7c94aa9c46...,call_0x1758476271d461dfb3c2d52709f787fbd6f5648...
2,0xc38920594ce7100b8859864e8df1c3d93ef3bbcaff4d...,93,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x0617d5ffb29c03ac35f1863b8a50ce1b52d446f6,0E-9,0x022c0d9f000000000000000000000000000000000000...,None,call,call,None,886843.0,NaN,1,"1,2",Reverted,0,2021-02-16 20:04:23+00:00,11869903,0xd4097f5b780518b89b2b40f4008a0dc9b38952793efc...,call_0xc38920594ce7100b8859864e8df1c3d93ef3bbc...
3,0x37e7a691816022afa5c0eaca2028eabe9f8a0af033ba...,14,0xb1f9ec02480dd9e16053b010dfc6e6c4b72ecad5,0x04fa0d235c4abf4bcf4787af4cf447de572ef828,0E-9,0x23b872dd0000000000000000000000004d246be90c2f...,None,call,call,None,664067.0,NaN,0,"0,7,0",Reverted,0,2021-02-16 15:50:37+00:00,11868753,0x19e9609fd72f42f36962602bb57a832d6377070ea05b...,call_0x37e7a691816022afa5c0eaca2028eabe9f8a0af...
4,0x05331da56d6579705256ed8ba259ba5b6b04d6f896e0...,116,0xfd09911130e6930bf87f2b0554c44f400bd80d3e,0xdc0c39780a6f3165f61fb27f3d47668033c898b5,0E-9,0xa9059cbb000000000000000000000000ac700361036d...,None,call,delegatecall,None,13942.0,NaN,0,"0,0,0,0",Out of gas,0,2021-02-16 10:28:55+00:00,11867275,0x9769a84f29eba585fa9bebfd37737ffcaec6c79e9454...,call_0x05331da56d6579705256ed8ba259ba5b6b04d6f...


In [19]:
# 하나의 transaction에 대해 여러 개의 trace가 남는다
trace_df.transaction_hash.value_counts().nlargest()

0xd8b8f2e6a402da98c278561d79a74702655727c06d366ead0796e6d1a2df4b72    1251
0xd911f266c4a0cbcf31e9c94dfe836100ca44d52a918bc3bc8d8f9996f1425445    1173
0x8975558ec884950c30efcb280025cbdae534bf2d892d721fb0fa5ebca34c1a59    1125
0x650542a0418807e4fbd5995176bf4b53ee44c219b4be4900316192eedac1c1cf    1119
0xac2ec718f30fcb4ccd9cda0252f67040f06b6b1092ea17b001e99743809c0b6f    1107
Name: transaction_hash, dtype: int64

In [20]:
trace_df[trace_df.transaction_hash == trace_df.transaction_hash.value_counts().nlargest(1).index[0]].head()

,transaction_hash,transaction_index,from_address,to_address,value,input,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,block_timestamp,block_number,block_hash,trace_id
4951,0xd8b8f2e6a402da98c278561d79a74702655727c06d36...,15.0,0xed2701f7135eab0d7ca02e6ab634ad6cbe159ffb,0x62de49fe8215dff88b9c1a2ea573e1471ff61f83,0E-9,0x9507d39a000000000000000000000000000000000000...,0xc8f0aa2a9a2d881caf0a041606ea43de06644d545ae0...,call,staticcall,None,3873513.0,4430.0,0,590,None,1,2021-02-16 04:33:27+00:00,11865672,0x15e00bce1a5d1d8d5769877829e44290f3132ab0f0bc...,call_0xd8b8f2e6a402da98c278561d79a74702655727c...
4952,0xd8b8f2e6a402da98c278561d79a74702655727c06d36...,15.0,0xed2701f7135eab0d7ca02e6ab634ad6cbe159ffb,0x62de49fe8215dff88b9c1a2ea573e1471ff61f83,0E-9,0x9507d39a000000000000000000000000000000000000...,0x63fb6e2243572d6d3757d43d62d3fbc954b26774b4c6...,call,staticcall,None,2000512.0,4430.0,0,920,None,1,2021-02-16 04:33:27+00:00,11865672,0x15e00bce1a5d1d8d5769877829e44290f3132ab0f0bc...,call_0xd8b8f2e6a402da98c278561d79a74702655727c...
4955,0xd8b8f2e6a402da98c278561d79a74702655727c06d36...,15.0,0xed2701f7135eab0d7ca02e6ab634ad6cbe159ffb,0x62de49fe8215dff88b9c1a2ea573e1471ff61f83,0E-9,0x9507d39a000000000000000000000000000000000000...,0x63fb6e2243572d6d3757d43d62d3fbc954b26774b4c6...,call,staticcall,None,3226537.0,4430.0,0,704,None,1,2021-02-16 04:33:27+00:00,11865672,0x15e00bce1a5d1d8d5769877829e44290f3132ab0f0bc...,call_0xd8b8f2e6a402da98c278561d79a74702655727c...
4957,0xd8b8f2e6a402da98c278561d79a74702655727c06d36...,15.0,0xed2701f7135eab0d7ca02e6ab634ad6cbe159ffb,0x62de49fe8215dff88b9c1a2ea573e1471ff61f83,0E-9,0x9507d39a000000000000000000000000000000000000...,0x00000000000000000000000000000000000000000000...,call,staticcall,None,3147755.0,4430.0,0,718,None,1,2021-02-16 04:33:27+00:00,11865672,0x15e00bce1a5d1d8d5769877829e44290f3132ab0f0bc...,call_0xd8b8f2e6a402da98c278561d79a74702655727c...
4965,0xd8b8f2e6a402da98c278561d79a74702655727c06d36...,15.0,0xed2701f7135eab0d7ca02e6ab634ad6cbe159ffb,0x62de49fe8215dff88b9c1a2ea573e1471ff61f83,0E-9,0x9507d39a000000000000000000000000000000000000...,0xdb0db5797eda61046725e183a49d784139df794d1344...,call,staticcall,None,6366576.0,4430.0,0,146,None,1,2021-02-16 04:33:27+00:00,11865672,0x15e00bce1a5d1d8d5769877829e44290f3132ab0f0bc...,call_0xd8b8f2e6a402da98c278561d79a74702655727c...


In [21]:
trace_df[trace_df.output.notna()]

,transaction_hash,transaction_index,from_address,to_address,value,input,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,block_timestamp,block_number,block_hash,trace_id
0,0xc79887b6be8189eac50bf079d75a6cdec067da8b8ed2...,99.0,0x06f05fecfc08b0c7c385f3a889876b84161be587,0xf6874c88757721a02f47592140905c4336dfbc61,0E-9,0xe5ab4da20000000000000000000000007186123dd914...,0x000000000000000000000000c19ca5e3c4f2117a6d63...,call,call,None,128416.0,83318.0,4,None,None,1,2021-02-16 00:52:10+00:00,11864669,0x6bc5925e1fba527a0fb06aefadd57718a02a281ae307...,call_0xc79887b6be8189eac50bf079d75a6cdec067da8...
1,0xfe8f508205890715d921caf4ae027998c81c9e446872...,297.0,0xf4ec738f334c63fd47d0fae60888fb6d7b1a439f,0xf6874c88757721a02f47592140905c4336dfbc61,0E-9,0xe5ab4da20000000000000000000000004f9dedc0ed08...,0x00000000000000000000000007338df0f2f687a0c07c...,call,call,None,128416.0,83318.0,4,None,None,1,2021-02-16 01:45:18+00:00,11864920,0x89a7b76f92cbfa22434f28bb2ae67d6ed3d6cdee9fbe...,call_0xfe8f508205890715d921caf4ae027998c81c9e4...
2,0x585db99cf2b135b282e4c12620b15112bb53edf9ae47...,192.0,0xd36329d2465b1e2da232de77342d3546156b9cc4,0xf6874c88757721a02f47592140905c4336dfbc61,0E-9,0xe5ab4da20000000000000000000000007186123dd914...,0x000000000000000000000000245311195d848800d50b...,call,call,None,128416.0,83318.0,4,None,None,1,2021-02-16 08:13:53+00:00,11866662,0xf1854bfe112e12685371b7d17d0c3bc4f5f71ca50b62...,call_0x585db99cf2b135b282e4c12620b15112bb53edf...
3,0xe29c5eef01e16ed8b1d69953c29e96dc3b867518e68a...,136.0,0x02e0690a5b0a4f8063bc9c427e14cd19372ea765,0xf6874c88757721a02f47592140905c4336dfbc61,0E-9,0xe5ab4da20000000000000000000000007186123dd914...,0x000000000000000000000000d3e40c4c92a1372e6bdc...,call,call,None,128416.0,83318.0,4,None,None,1,2021-02-16 04:21:48+00:00,11865619,0xb4c1f51d6e5da514e6a3376b6529dd311ffc4e6045f7...,call_0xe29c5eef01e16ed8b1d69953c29e96dc3b86751...
4,0x2b772a5be627bb21034d9dbf6272f1d9ee9e66cba64d...,43.0,0xe4a4a8193899031f9206d981e2f0e6862086893d,0xf6874c88757721a02f47592140905c4336dfbc61,0E-9,0xe5ab4da20000000000000000000000007186123dd914...,0x0000000000000000000000000af94b09cb3ffaa0f3e8...,call,call,None,128416.0,83318.0,4,None,None,1,2021-02-16 02:29:39+00:00,11865099,0xe6f379653db35d317eb922ab195b5f3e9ed8b99bca5d...,call_0x2b772a5be627bb21034d9dbf6272f1d9ee9e66c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4226249,0x159d1cc51fc299e76c597964a63bbbf48bcf05d5e335...,119.0,0x46f5c799890573808ce0b59febee07c1d34c93ef,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0E-9,0x18cbafe5000000000000000000000000000000000000...,0x00000000000000000000000000000000000000000000...,call,call,None,145180.0,128006.0,5,None,None,1,2021-02-16 13:46:45+00:00,11868195,0x3d55d69c358974e67439cab100f10a46e28dd960c2b7...,call_0x159d1cc51fc299e76c597964a63bbbf48bcf05d...
4226250,0x78983d6f079f3b5097cf5506eb9dcc0ab7a60a215290...,240.0,0x663b4cd739a1a0d6eee393a22dd2a7c02d1322ac,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0E-9,0x18cbafe5000000000000000000000000000000000000...,0x00000000000000000000000000000000000000000000...,call,call,None,146311.0,129030.0,5,None,None,1,2021-02-16 15:51:58+00:00,11868757,0x65633d8b155fe2c291de58ce179a89058dbca5ae044e...,call_0x78983d6f079f3b5097cf5506eb9dcc0ab7a60a2...
4226251,0x4bd8ae2298ddd388aa6a74c3ee0dfd7a7d32a1d8d6c7...,9.0,0x8c4b7870fc7dff2cb1e854858533ceddaf3eebf4,0x7fd43779d589914b1f25c4d2e55b619d81d1e1bc,0E-9,0x391252150000000000000000000000006e36c2a0aa63...,0x,call,call,None,156023.0,86023.0,1,None,None,1,2021-02-16 12:45:47+00:00,11867909,0x22b179af63b9859b01020bca83a4cb880bbf7ea82a2c...,call_0x4bd8ae2298ddd388aa6a74c3ee0dfd7a7d32a1d...
4226252,0x229189ed7258d3feb4ddcc0a771cd3d330e8e8446db4...,119.0,0x00000000c0293c8ca34dac9bcc0f953532d34e4d,0xd1ceeeeee83f8bcf3bedad437202b6154e9f5405,0E-9,0xca722cdc47a010a1d62c633438220ca69bebcaddea08...,0x,call,call,None,42912.0,25863.0,1,None,None,1,2021-02-16 22:28:18+00:00,11870540,0xea87ca

[trace_type 참조](https://medium.com/chainalysis/ethereum-traces-not-transactions-3f0533d26aa)
- **call**: Used to transfer ether from one account to another and/or to call a smart contract function defined by parameters in the data field. This trace also encompasses delegatecall and callcode.
- **create**: Used to create a smart contract, and ether is transferred to the newly created smart contract
- **suicide**: Used by an owner of a smart contract to kill the smart contract. Triggers a transfer of ether for a refund for killing a contract. Additionally, killing a smart contract can free memory in the blockchain, which can also affects the value transferred.


In [22]:
trace_df.trace_type.value_counts()

call       4163733
suicide      28975
create       26779
reward        6767
Name: trace_type, dtype: int64

[call_type:delegatecall 참조](https://steemit.com/kr/@etainclub/4-delegatecall)
- 일반적 컨트랙트에서 외부 메시지 호출은 CALL 함수에 의해 처리되고, 코드는 외부 컨트랙트에서 실행되게 됨
- DELEGATECALL은 CALL과 거의 같은 기능인데, 코드가 외부 컨트랙트에서 실행되는게 아니라 호출한 컨트랙트에서 실행됨.

[call_type:delegatecall 참조](https://medium.com/day34/solidity-0-5-0-%EC%97%90%EC%84%9C%EC%9D%98-%EB%B3%80%EA%B2%BD%EC%82%AC%ED%95%AD%EC%9D%84-%EC%86%8C%EA%B0%9C%ED%95%A9%EB%8B%88%EB%8B%A4-ab6104296164)
- 솔리디티 공식 문서에 따르면 EVM 레벨에서 상태값이 바뀌지 않는 것을 보장하기 위해 STATICCALL opcode 를 새롭게 추가
- 쉽게 얘기하자면 함수 내에서 상태값을 바꾸지 않는 pure 함수와 view 함수는 staticcall 로 호출한다고 생각하면 된다

In [23]:
trace_df.call_type.value_counts()

call            2611167
staticcall      1181668
delegatecall     370898
Name: call_type, dtype: int64

[uncle 보상 참조](https://taeyonghwang.github.io/ethereum/uncle-reward/)

In [24]:
trace_df.reward_type.value_counts()

block    6485
uncle     282
Name: reward_type, dtype: int64

In [32]:
print(f"gas>0: {100*(trace_df.gas>0).mean():.2f}%")
print(f"mean gas: {trace_df.loc[trace_df.gas > 0, 'gas'].mean():.2f}")

gas>0: 91.45%
mean gas: 304682.10


In [34]:
print(f"gas used > 0: {100*(trace_df.gas_used>0).mean():.2f}%")
print(f"mean gas used: {trace_df.loc[trace_df.gas_used > 0, 'gas_used'].mean():.2f}")

gas used > 0: 84.70%
mean gas used: 36321.83


In [38]:
trace_df.subtraces.value_counts(normalize=True).sort_index().head(10)

0    0.712531
1    0.147553
2    0.028120
3    0.056511
4    0.017066
5    0.019752
6    0.007140
7    0.005277
8    0.001254
9    0.001272
Name: subtraces, dtype: float64

In [40]:
trace_df.loc[trace_df.trace_address.notna(), 'trace_address']

49                     2,0,0
50                   0,4,2,0
51                    1,1,11
52                   0,1,4,0
53         1,0,2,0,1,0,1,1,5
                 ...        
4091852           0,3,0,15,1
4091853                4,4,0
4091854               2,10,2
4091855                  6,0
4091856            0,4,0,0,0
Name: trace_address, Length: 3001571, dtype: object

[가스 에러 참조](https://playdev.medium.com/%EC%9D%B4%EB%8D%94%EB%A6%AC%EC%9B%80-%EA%B0%80%EC%8A%A4%EC%97%90-%EB%8C%80%ED%95%9C-%EC%9D%B4%ED%95%B4-gas-price-gas-limit-block-gas-limit-161268a5fc54)

In [43]:
trace_df.error.value_counts()

Reverted                46308
Out of gas               4801
Bad instruction          1597
Bad jump destination      322
Name: error, dtype: int64

In [47]:
# 0이면 에러다
trace_df.status.value_counts()

1    4088354
0     137900
Name: status, dtype: int64

In [46]:
trace_df.loc[trace_df.status==1, 'error'].value_counts()

Series([], Name: error, dtype: int64)

In [50]:
trace_df.block_timestamp.min()

Timestamp('2021-02-16 00:00:17+0000', tz='UTC')

In [51]:
trace_df.block_timestamp.max()

Timestamp('2021-02-16 23:59:56+0000', tz='UTC')

### token_transfers

In [56]:
%%bigquery transfer_df
select *
from `bigquery-public-data.crypto_ethereum.token_transfers`
where DATE(block_timestamp) = "2021-02-16"

Downloading: 100%|██████████| 823929/823929 [00:48<00:00, 17000.96rows/s]


In [66]:
transfer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 823929 entries, 0 to 823928
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype              
---  ------            --------------   -----              
 0   token_address     823929 non-null  object             
 1   from_address      823929 non-null  object             
 2   to_address        823929 non-null  object             
 3   value             823929 non-null  object             
 4   transaction_hash  823929 non-null  object             
 5   log_index         823929 non-null  int64              
 6   block_timestamp   823929 non-null  datetime64[ns, UTC]
 7   block_number      823929 non-null  int64              
 8   block_hash        823929 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(2), object(6)
memory usage: 56.6+ MB


In [59]:
transfer_df.token_address.value_counts().head()

0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2    152477
0xdac17f958d2ee523a2206206994597c13d831ec7    143607
0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48     55572
0x514910771af9ca656af840dff83e8264ecf986ca     20414
0x6b175474e89094c44da98b954eedeac495271d0f     17145
Name: token_address, dtype: int64

In [61]:
target_hash = transfer_df.transaction_hash.sample().values[0]

In [62]:
transfer_df[transfer_df.transaction_hash==target_hash]

,token_address,from_address,to_address,value,transaction_hash,log_index,block_timestamp,block_number,block_hash
680272,0x40449d1f4c2d4f88dfd5b18868c76738a4e52fd4,0xde9a570afeca6f1fefc7edc7c41c8aebc5eb5b65,0xbcf39f0edda668c58371e519af37ca705f2bfcbd,1262500000000000000,0x52170f2b5a142b6c874a8327e2ce9fa3c0b38fd31c07...,140,2021-02-16 09:46:05+00:00,11867090,0x8a16fb5d0020a75005f3085a27e8f53ad3546a1cdc49...


In [63]:
trace_df[trace_df.transaction_hash==target_hash]

,transaction_hash,transaction_index,from_address,to_address,value,input,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,block_timestamp,block_number,block_hash,trace_id
604993,0x52170f2b5a142b6c874a8327e2ce9fa3c0b38fd31c07...,63.0,0xde9a570afeca6f1fefc7edc7c41c8aebc5eb5b65,0xbcf39f0edda668c58371e519af37ca705f2bfcbd,0E-9,0xe2bbb158000000000000000000000000000000000000...,0x,call,call,None,168884.0,105451.0,2,None,None,1,2021-02-16 09:46:05+00:00,11867090,0x8a16fb5d0020a75005f3085a27e8f53ad3546a1cdc49...,call_0x52170f2b5a142b6c874a8327e2ce9fa3c0b38fd...
1058945,0x52170f2b5a142b6c874a8327e2ce9fa3c0b38fd31c07...,63.0,0xbcf39f0edda668c58371e519af37ca705f2bfcbd,0x40449d1f4c2d4f88dfd5b18868c76738a4e52fd4,0E-9,0x70a08231000000000000000000000000de9a570afeca...,0x00000000000000000000000000000000000000000000...,call,staticcall,None,161531.0,1180.0,0,0,None,1,2021-02-16 09:46:05+00:00,11867090,0x8a16fb5d0020a75005f3085a27e8f53ad3546a1cdc49...,call_0x52170f2b5a142b6c874a8327e2ce9fa3c0b38fd...
1637608,0x52170f2b5a142b6c874a8327e2ce9fa3c0b38fd31c07...,63.0,0xbcf39f0edda668c58371e519af37ca705f2bfcbd,0x40449d1f4c2d4f88dfd5b18868c76738a4e52fd4,0E-9,0x23b872dd000000000000000000000000de9a570afeca...,0x00000000000000000000000000000000000000000000...,call,call,None,158668.0,21449.0,0,1,None,1,2021-02-16 09:46:05+00:00,11867090,0x8a16fb5d0020a75005f3085a27e8f53ad3546a1cdc49...,call_0x52170f2b5a142b6c874a8327e2ce9fa3c0b38fd...


### transactions

- [참조 1](https://brownbears.tistory.com/385#:~:text=%ED%8A%B8%EB%9E%9C%EC%9E%AD%EC%85%98%EC%9D%80%20%ED%95%98%EB%82%98%EC%9D%98%20%EC%84%9C%EB%AA%85,%EB%A5%BC%20%EC%9D%B4%EC%9A%A9%ED%95%98%EC%97%AC%20%EC%84%9C%EB%AA%85%EB%90%A9%EB%8B%88%EB%8B%A4.)
- [참조 2](https://m.post.naver.com/viewer/postView.nhn?volumeNo=16696266&memberNo=41484364)
- [참조 스팀 블로그 연재](https://steemit.com/kr/@skt1/34-1-transaction)

In [64]:
%%bigquery transaction_df
select *
from `bigquery-public-data.crypto_ethereum.transactions`
where DATE(block_timestamp) = "2021-02-16"

Downloading: 100%|██████████| 1217916/1217916 [02:08<00:00, 9509.71rows/s] 


In [65]:
transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1217916 entries, 0 to 1217915
Data columns (total 17 columns):
 #   Column                       Non-Null Count    Dtype              
---  ------                       --------------    -----              
 0   hash                         1217916 non-null  object             
 1   nonce                        1217916 non-null  int64              
 2   transaction_index            1217916 non-null  int64              
 3   from_address                 1217916 non-null  object             
 4   to_address                   1215416 non-null  object             
 5   value                        1217916 non-null  object             
 6   gas                          1217916 non-null  int64              
 7   gas_price                    1217916 non-null  int64              
 8   input                        1217916 non-null  object             
 9   receipt_cumulative_gas_used  1217916 non-null  int64              
 10  receipt_gas_used  

In [67]:
transaction_df.head()

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash
0,0x9bed29207a8fac26a304eedebec9978a7c82dc27fcb9...,1,265,0x952ecc28466db64aa40cf6d032acad445cffa93e,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,965284552877856027.000000000,42000,197000000000,0x,10538495,21000,None,None,1,2021-02-16 18:30:43+00:00,11869486,0x89de221285ce03555213a8855434b869e941f56161dd...
1,0xf28ff5d1ce91329575890e4a0ce3cee59f0edc339b3f...,6,114,0x324e0f076c28875b71f949eec960b013702c4aab,0x6595b8fd9c920c81500dca94e53cdc712513fb1f,0E-9,52192,115000000000,0x095ea7b30000000000000000000000007a250d5630b4...,5707949,27934,None,None,1,2021-02-16 13:50:46+00:00,11868220,0x5eab64b78f4810f011492786619a97e3f3590b5e0aa0...
2,0x72f6d9462056e82ad249620788e0eedfee83b175629c...,33,159,0x2abb000c0544152ab12e39549c9457410f206790,0x881d40237659c251811cec9c364ef91dc08d300c,2100000000000000000.000000000,260128,155000000000,0x5f575529000000000000000000000000000000000000...,7844200,172060,None,None,1,2021-02-16 04:31:58+00:00,11865665,0xc3875b150f58ca8dfa86cc27f48c69d09d1517760437...
3,0x2d0c51d008e8036f7d9748d82cc123519c7c6b530344...,783,173,0xe3bdda6413313ed58585e47441815e9662bdd818,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0E-9,126082,186000000000,0x38ed1739000000000000000000000000000000000000...,7200917,108470,None,None,1,2021-02-16 12:15:00+00:00,11867745,0x1e22c7163a1a657099a74746410792aae0206b3cec7f...
4,0x6e6ec466971428576dc66816def76ae87772f63e0ea3...,9957144,170,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,0xa0b73e1ff0b80914ab6fe0444e65848c4c34450b,0E-9,150000,170400000000,0xa9059cbb0000000000000000000000001c989aa4f891...,10981096,53461,None,None,1,2021-02-16 15:16:37+00:00,11868604,0x7401c3d3369dec77af5595b897db9f94f7319388dbf1...
